## TODO
- Cross-check protocol costs once they are fixed in the paper.
- Fill in the correct costs for subtractive sets i.e. size, expansion factor, inverse expansion factor, etc.

In [1]:
from symbolic_variables import *
from gen_params import gen_ring_param, gen_sis_param, gen_sis_param_aggresive
from ring_params import *
from protocol import Protocol

In [17]:
## Big rings lead to large proof size

f = 27720
log_q = 100
ring_param = {
  'f': f,
  'fhat': f/2,
 'phi': euler_phi(f),
 'q': 2 ^ log_q,
 'log_q': log_q,
 'Rq_size': log_q * euler_phi(f),
 'e': 2,
 'C': f / 11,
 'gamma_2': 1,
 'theta_2': ceil(f/(2 * sqrt(2))),
 'gamma_inf': 2,
 'theta_inf': f,
 'ring_exp_inf': ceil(f/(2 * sqrt(2)))}
sis_param = gen_sis_param(ring_param = ring_param, log_beta_sis = 60)
rep = 1
wit_rdim = 2**30/ring_param["phi"]
beta_inf = 1
beta_2 = sqrt(wit_rdim * ring_param["phi"] * ring_param["fhat"]) * beta_inf
tdim = 20
ell = 2
print(sis_param)


protocol = Protocol(
    ring_param = ring_param,
    sis_param = sis_param,
    rep = rep, 
    wit_rdim = wit_rdim,  
    beta_2 = beta_2,
    beta_inf = beta_inf
)

loop = [("bdecomp", {"ell":ell}), ("split", {"tdim":tdim}), ("fold", {"repout":1}), ("norm", {}), ("batch", {})]
ops = 3 * loop + [("finish", {})]

for op, params in ops:
    protocol.execute(op, **params)

protocol.execute("finish")
protocol.extract()
protocol.print()

{'n': 1, 'log_beta_sis': 60}
Global parameters:


conductor,ring degree,subtractive set size,log modulus,log sis norm,Rq size
\({\mathfrak{f}}\),\({\varphi}\),\({|\mathcal{C}|}\),\(\log\left({q}\right)\),\(\log\left({\beta_{\mathsf{sis}}}\right)\),\({\mathcal{R}_{q}}\)
\(27720\),\(5760\),\(2520\),\(100\),\(60\),70 KB



Protocol history:


operation,#relations,witness dimension,bundle size,log witness norm can-2,log extration norm can-2,log witness norm coeff-inf,log extration norm coeff-inf,prover comm,witness size,soundness
,"\(\left({n'}, {n''}\right)\)",\({m}\),\({r}\),"\(\log\left({\beta_{\mathsf{wit},2}}\right)\)","\(\log\left({\beta_{\mathsf{ext},2}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{wit},\infty}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{ext},\infty}}\right)\)",\({|\pi|}\),\({|w|}\),\(-\log\left({\kappa}\right)\)
init,"\(\left(1, 0\right)\)",\(\frac{8388608}{45}\),\(1\),\(22\),\(49\),\(1\),\(46\),0 B,128 MB,\(+\infty\)
bdecomp,"\(\left(1, 0\right)\)",\(\frac{8388608}{45}\),\(2\),\(23\),\(49\),\(1\),\(46\),70 KB,256 MB,\(+\infty\)
split,"\(\left(1, 0\right)\)",\(\frac{2097152}{225}\),\(40\),\(23\),\(47\),\(1\),\(44\),2 MB,256 MB,\(+\infty\)
fold,"\(\left(1, 0\right)\)",\(\frac{2097152}{225}\),\(1\),\(28\),\(45\),\(2\),\(44\),5 MB,12 MB,\(6\)
norm,"\(\left(1, 3\right)\)",\(\frac{2097152}{225}\),\(14\),\(28\),\(28\),\(2\),\(28\),9 MB,179 MB,\(6\)
batch,"\(\left(1, 1\right)\)",\(\frac{2097152}{225}\),\(14\),\(28\),\(55\),\(2\),\(50\),9 MB,179 MB,\(6\)
bdecomp,"\(\left(1, 1\right)\)",\(\frac{2097152}{225}\),\(28\),\(23\),\(55\),\(1\),\(50\),11 MB,179 MB,\(6\)
split,"\(\left(1, 1\right)\)",\(\frac{524288}{1125}\),\(560\),\(23\),\(53\),\(1\),\(48\),84 MB,179 MB,\(6\)
fold,"\(\left(1, 1\right)\)",\(\frac{524288}{1125}\),\(1\),\(32\),\(51\),\(2\),\(48\),122 MB,655 KB,\(3\)


In [6]:
## Strategy: 
##   - Opener: init->norm->batch->split->fold
##   - Loop: norm->batch->bdecomp->split->fold

ring_param = ring_param_60_16
sis_param = gen_sis_param(ring_param = ring_param, log_beta_sis = 42)
rep = 25
wit_rdim = 2**30/ring_param["phi"]/rep
beta_inf = 1
beta_2 = sqrt(wit_rdim * ring_param["phi"] * ring_param["fhat"]) * beta_inf
ell = 2
tdim = 4

protocol = Protocol(
    ring_param = ring_param,
    sis_param = sis_param,
    rep = rep, 
    wit_rdim = wit_rdim,  
    beta_2 = beta_2,
    beta_inf = beta_inf
)

opener = [("norm", {}), ("batch", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("norm", {}), ("batch", {}), ("bdecomp", {"ell":ell}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener + 3 * loop + [("finish", {})]

for op, params in ops:
    protocol.execute(op, **params)
protocol.extract()
protocol.print()

Global parameters:


conductor,ring degree,subtractive set size,log modulus,log sis norm,Rq size
\({\mathfrak{f}}\),\({\varphi}\),\({|\mathcal{C}|}\),\(\log\left({q}\right)\),\(\log\left({\beta_{\mathsf{sis}}}\right)\),\({\mathcal{R}_{q}}\)
\(60\),\(16\),\(12\),\(64\),\(42\),128 B



Protocol history:


operation,#relations,witness dimension,bundle size,log witness norm can-2,log extration norm can-2,log witness norm coeff-inf,log extration norm coeff-inf,prover comm,witness size,soundness
,"\(\left({n'}, {n''}\right)\)",\({m}\),\({r}\),"\(\log\left({\beta_{\mathsf{wit},2}}\right)\)","\(\log\left({\beta_{\mathsf{ext},2}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{wit},\infty}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{ext},\infty}}\right)\)",\({|\pi|}\),\({|w|}\),\(-\log\left({\kappa}\right)\)
init,"\(\left(68, 0\right)\)",\(\frac{67108864}{25}\),\(25\),\(16\),\(16\),\(1\),\(16\),0 B,128 MB,\(+\infty\)
norm,"\(\left(68, 3\right)\)",\(\frac{67108864}{25}\),\(41\),\(18\),\(16\),\(1\),\(16\),151 KB,209 MB,\(106\)
batch,"\(\left(68, 1\right)\)",\(\frac{67108864}{25}\),\(41\),\(18\),\(37\),\(1\),\(32\),151 KB,209 MB,\(106\)
split,"\(\left(68, 1\right)\)",\(\frac{16777216}{25}\),\(164\),\(18\),\(37\),\(1\),\(32\),1 MB,209 MB,\(106\)
fold,"\(\left(68, 1\right)\)",\(\frac{16777216}{25}\),\(25\),\(27\),\(36\),\(2\),\(32\),1 MB,64 MB,\(83\)
norm,"\(\left(68, 4\right)\)",\(\frac{16777216}{25}\),\(36\),\(27\),\(27\),\(2\),\(27\),1 MB,92 MB,\(83\)
batch,"\(\left(68, 1\right)\)",\(\frac{16777216}{25}\),\(36\),\(27\),\(41\),\(2\),\(36\),1 MB,92 MB,\(83\)
bdecomp,"\(\left(68, 1\right)\)",\(\frac{16777216}{25}\),\(72\),\(18\),\(41\),\(1\),\(36\),2 MB,92 MB,\(83\)
split,"\(\left(68, 1\right)\)",\(\frac{4194304}{25}\),\(288\),\(18\),\(39\),\(1\),\(34\),3 MB,92 MB,\(83\)


In [7]:
## Strategy: 
##   - Opener: init->norm->batch->split->fold
##   - Loop: norm->batch->bdecomp->split->fold

ring_param = ring_param_24_8
sis_param = gen_sis_param(ring_param = ring_param, log_beta_sis = 41)
rep = 30
wit_rdim = 2**30/ring_param["phi"]/rep
beta_inf = 1
beta_2 = sqrt(wit_rdim * ring_param["phi"] * ring_param["fhat"]) * beta_inf
ell = 2
tdim = 4

protocol = Protocol(
    ring_param = ring_param,
    sis_param = sis_param,
    rep = rep, 
    wit_rdim = wit_rdim,  
    beta_2 = beta_2,
    beta_inf = beta_inf
)

opener = [("norm", {}), ("batch", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("norm", {}), ("batch", {}), ("bdecomp", {"ell":ell}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener + 3 * loop + [("finish", {})]

for op, params in ops:
    protocol.execute(op, **params)
protocol.extract()
protocol.print()

Global parameters:


conductor,ring degree,subtractive set size,log modulus,log sis norm,Rq size
\({\mathfrak{f}}\),\({\varphi}\),\({|\mathcal{C}|}\),\(\log\left({q}\right)\),\(\log\left({\beta_{\mathsf{sis}}}\right)\),\({\mathcal{R}_{q}}\)
\(24\),\(8\),\(8\),\(64\),\(41\),64 B



Protocol history:


operation,#relations,witness dimension,bundle size,log witness norm can-2,log extration norm can-2,log witness norm coeff-inf,log extration norm coeff-inf,prover comm,witness size,soundness
,"\(\left({n'}, {n''}\right)\)",\({m}\),\({r}\),"\(\log\left({\beta_{\mathsf{wit},2}}\right)\)","\(\log\left({\beta_{\mathsf{ext},2}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{wit},\infty}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{ext},\infty}}\right)\)",\({|\pi|}\),\({|w|}\),\(-\log\left({\kappa}\right)\)
init,"\(\left(130, 0\right)\)",\(\frac{67108864}{15}\),\(30\),\(15\),\(15\),\(1\),\(15\),0 B,128 MB,\(+\infty\)
norm,"\(\left(130, 3\right)\)",\(\frac{67108864}{15}\),\(46\),\(17\),\(15\),\(1\),\(15\),138 KB,196 MB,\(105\)
batch,"\(\left(130, 1\right)\)",\(\frac{67108864}{15}\),\(46\),\(17\),\(36\),\(1\),\(31\),138 KB,196 MB,\(105\)
split,"\(\left(130, 1\right)\)",\(\frac{16777216}{15}\),\(184\),\(17\),\(36\),\(1\),\(31\),1 MB,196 MB,\(105\)
fold,"\(\left(130, 1\right)\)",\(\frac{16777216}{15}\),\(30\),\(27\),\(35\),\(2\),\(31\),1 MB,64 MB,\(83\)
norm,"\(\left(130, 4\right)\)",\(\frac{16777216}{15}\),\(41\),\(27\),\(27\),\(2\),\(27\),1 MB,87 MB,\(83\)
batch,"\(\left(130, 1\right)\)",\(\frac{16777216}{15}\),\(41\),\(27\),\(40\),\(2\),\(34\),1 MB,87 MB,\(83\)
bdecomp,"\(\left(130, 1\right)\)",\(\frac{16777216}{15}\),\(82\),\(18\),\(40\),\(1\),\(34\),1 MB,87 MB,\(83\)
split,"\(\left(130, 1\right)\)",\(\frac{4194304}{15}\),\(328\),\(18\),\(38\),\(1\),\(32\),3 MB,87 MB,\(83\)


In [8]:
## Strategy: 
##   - Opener: init->norm->batch->split->fold
##   - Loop: bdecomp->norm->batch->split->fold

ring_param = ring_param_60_16
sis_param = gen_sis_param(ring_param = ring_param, log_beta_sis = 38)
rep = 25
wit_rdim = 2**30/ring_param["phi"]/rep
beta_inf = 1
beta_2 = sqrt(wit_rdim * ring_param["phi"] * ring_param["fhat"]) * beta_inf
ell = 2
base = 2**5+1
tdim = 6

protocol = Protocol(
    ring_param = ring_param,
    sis_param = sis_param,
    rep = rep, 
    wit_rdim = wit_rdim,  
    beta_2 = beta_2,
    beta_inf = beta_inf
)

opener = [("norm", {}), ("batch", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"ell":ell}), ("norm", {}), ("batch", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener + 3 * loop + [("finish", {})]

for op, params in ops:
    protocol.execute(op, **params)
protocol.extract()
protocol.print()

Global parameters:


conductor,ring degree,subtractive set size,log modulus,log sis norm,Rq size
\({\mathfrak{f}}\),\({\varphi}\),\({|\mathcal{C}|}\),\(\log\left({q}\right)\),\(\log\left({\beta_{\mathsf{sis}}}\right)\),\({\mathcal{R}_{q}}\)
\(60\),\(16\),\(12\),\(64\),\(38\),128 B



Protocol history:


operation,#relations,witness dimension,bundle size,log witness norm can-2,log extration norm can-2,log witness norm coeff-inf,log extration norm coeff-inf,prover comm,witness size,soundness
,"\(\left({n'}, {n''}\right)\)",\({m}\),\({r}\),"\(\log\left({\beta_{\mathsf{wit},2}}\right)\)","\(\log\left({\beta_{\mathsf{ext},2}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{wit},\infty}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{ext},\infty}}\right)\)",\({|\pi|}\),\({|w|}\),\(-\log\left({\kappa}\right)\)
init,"\(\left(56, 0\right)\)",\(\frac{67108864}{25}\),\(25\),\(16\),\(16\),\(1\),\(16\),0 B,128 MB,\(+\infty\)
norm,"\(\left(56, 3\right)\)",\(\frac{67108864}{25}\),\(41\),\(18\),\(16\),\(1\),\(16\),127 KB,209 MB,\(106\)
batch,"\(\left(56, 1\right)\)",\(\frac{67108864}{25}\),\(41\),\(18\),\(30\),\(1\),\(25\),127 KB,209 MB,\(106\)
split,"\(\left(56, 1\right)\)",\(\frac{33554432}{75}\),\(246\),\(18\),\(30\),\(1\),\(25\),1 MB,209 MB,\(106\)
fold,"\(\left(56, 1\right)\)",\(\frac{33554432}{75}\),\(25\),\(28\),\(29\),\(2\),\(25\),2 MB,42 MB,\(82\)
bdecomp,"\(\left(56, 1\right)\)",\(\frac{33554432}{75}\),\(50\),\(18\),\(20\),\(1\),\(20\),2 MB,42 MB,\(82\)
norm,"\(\left(56, 4\right)\)",\(\frac{33554432}{75}\),\(65\),\(18\),\(18\),\(1\),\(18\),2 MB,55 MB,\(82\)
batch,"\(\left(56, 1\right)\)",\(\frac{33554432}{75}\),\(65\),\(18\),\(29\),\(1\),\(23\),2 MB,55 MB,\(82\)
split,"\(\left(56, 1\right)\)",\(\frac{16777216}{225}\),\(390\),\(18\),\(29\),\(1\),\(23\),4 MB,55 MB,\(82\)


In [9]:
## Strategy: 
##   - Opener: init->norm->batch->split->fold
##   - Loop: bdecomp->norm->batch->split->fold

ring_param = ring_param_24_8
sis_param = gen_sis_param(ring_param = ring_param, log_beta_sis = 34)
rep = 30
wit_rdim = 2**30/ring_param["phi"]/rep
beta_inf = 1
beta_2 = sqrt(wit_rdim * ring_param["phi"] * ring_param["fhat"]) * beta_inf
ell = 2
tdim = 2

protocol = Protocol(
    ring_param = ring_param,
    sis_param = sis_param,
    rep = rep, 
    wit_rdim = wit_rdim,  
    beta_2 = beta_2,
    beta_inf = beta_inf
)

opener = [("norm", {}), ("batch", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"ell":ell}), ("norm", {}), ("batch", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener + 7 * loop + [("finish", {})]

for op, params in ops:
    protocol.execute(op, **params)
protocol.extract()
protocol.print()

Global parameters:


conductor,ring degree,subtractive set size,log modulus,log sis norm,Rq size
\({\mathfrak{f}}\),\({\varphi}\),\({|\mathcal{C}|}\),\(\log\left({q}\right)\),\(\log\left({\beta_{\mathsf{sis}}}\right)\),\({\mathcal{R}_{q}}\)
\(24\),\(8\),\(8\),\(64\),\(34\),64 B



Protocol history:


operation,#relations,witness dimension,bundle size,log witness norm can-2,log extration norm can-2,log witness norm coeff-inf,log extration norm coeff-inf,prover comm,witness size,soundness
,"\(\left({n'}, {n''}\right)\)",\({m}\),\({r}\),"\(\log\left({\beta_{\mathsf{wit},2}}\right)\)","\(\log\left({\beta_{\mathsf{ext},2}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{wit},\infty}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{ext},\infty}}\right)\)",\({|\pi|}\),\({|w|}\),\(-\log\left({\kappa}\right)\)
init,"\(\left(90, 0\right)\)",\(\frac{67108864}{15}\),\(30\),\(15\),\(15\),\(1\),\(15\),0 B,128 MB,\(+\infty\)
norm,"\(\left(90, 3\right)\)",\(\frac{67108864}{15}\),\(46\),\(17\),\(15\),\(1\),\(15\),98 KB,196 MB,\(105\)
batch,"\(\left(90, 1\right)\)",\(\frac{67108864}{15}\),\(46\),\(17\),\(28\),\(1\),\(24\),98 KB,196 MB,\(105\)
split,"\(\left(90, 1\right)\)",\(\frac{33554432}{15}\),\(92\),\(17\),\(28\),\(1\),\(24\),360 KB,196 MB,\(105\)
fold,"\(\left(90, 1\right)\)",\(\frac{33554432}{15}\),\(30\),\(26\),\(27\),\(2\),\(24\),532 KB,128 MB,\(84\)
bdecomp,"\(\left(90, 1\right)\)",\(\frac{33554432}{15}\),\(60\),\(18\),\(20\),\(1\),\(20\),703 KB,128 MB,\(84\)
norm,"\(\left(90, 4\right)\)",\(\frac{33554432}{15}\),\(76\),\(18\),\(18\),\(1\),\(18\),808 KB,162 MB,\(84\)
batch,"\(\left(90, 1\right)\)",\(\frac{33554432}{15}\),\(76\),\(18\),\(28\),\(1\),\(23\),808 KB,162 MB,\(84\)
split,"\(\left(90, 1\right)\)",\(\frac{16777216}{15}\),\(152\),\(18\),\(28\),\(1\),\(23\),1 MB,162 MB,\(84\)


In [71]:
## Strategy: 
##   - Opener: init->norm->batch->split->fold
##   - Loop: bdecomp->norm->batch->split->fold
##   - finish: bdecomp->norm
log_q = 32

ring_param = ring_param_24_8 = {'f': 24,
 'fhat': 12,
 'phi': 8,
 'q': 2^log_q,
 'log_q': log_q,
 'Rq_size': 8 * log_q,
 'e': 4,
 'C': 3,
 'gamma_2': 1,
 'theta_2': 8,
 'gamma_inf': 2,
 'theta_inf': 8,
 'ring_exp_inf': 8}


sis_param = gen_sis_param(ring_param = ring_param, log_beta_sis = 29)
rep = 57
wit_rdim = 2**30/ring_param["phi"]/rep
beta_inf = 1
beta_2 = sqrt(wit_rdim * ring_param["phi"] * ring_param["fhat"]) * beta_inf
ell = 2
tdim = 2

protocol = Protocol(
    ring_param = ring_param,
    sis_param = sis_param,
    rep = rep, 
    wit_rdim = wit_rdim,  
    beta_2 = beta_2,
    beta_inf = beta_inf
)

opener = [("norm", {}), ("batch", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"ell":ell}), ("norm", {}), ("batch", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener + 5 * loop + [("bdecomp", {"ell":ell}), ("norm", {}), ("finish", {})]

for op, params in ops:
    protocol.execute(op, **params)
protocol.extract()
protocol.print()

Global parameters:


conductor,ring degree,subtractive set size,log modulus,log sis norm,Rq size
\({\mathfrak{f}}\),\({\varphi}\),\({|\mathcal{C}|}\),\(\log\left({q}\right)\),\(\log\left({\beta_{\mathsf{sis}}}\right)\),\({\mathcal{R}_{q}}\)
\(24\),\(8\),\(3\),\(32\),\(29\),32 B



Protocol history:


operation,#relations,witness dimension,bundle size,log witness norm can-2,log extration norm can-2,log witness norm coeff-inf,log extration norm coeff-inf,prover comm,witness size,soundness
,"\(\left({n'}, {n''}\right)\)",\({m}\),\({r}\),"\(\log\left({\beta_{\mathsf{wit},2}}\right)\)","\(\log\left({\beta_{\mathsf{ext},2}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{wit},\infty}}\right)\)","\(\log\left(2 \, {\beta_{\mathsf{ext},\infty}}\right)\)",\({|\pi|}\),\({|w|}\),\(-\log\left({\kappa}\right)\)
init,"\(\left(130, 0\right)\)",\(\frac{134217728}{57}\),\(57\),\(14\),\(14\),\(1\),\(14\),0 B,128 MB,\(+\infty\)
norm,"\(\left(130, 3\right)\)",\(\frac{134217728}{57}\),\(73\),\(16\),\(14\),\(1\),\(14\),71 KB,163 MB,\(106\)
batch,"\(\left(130, 1\right)\)",\(\frac{134217728}{57}\),\(73\),\(16\),\(28\),\(1\),\(24\),71 KB,163 MB,\(106\)
split,"\(\left(130, 1\right)\)",\(\frac{67108864}{57}\),\(146\),\(16\),\(28\),\(1\),\(24\),370 KB,163 MB,\(106\)
fold,"\(\left(130, 1\right)\)",\(\frac{67108864}{57}\),\(57\),\(27\),\(27\),\(2\),\(24\),630 KB,128 MB,\(84\)
bdecomp,"\(\left(130, 1\right)\)",\(\frac{67108864}{57}\),\(114\),\(18\),\(20\),\(1\),\(20\),864 KB,128 MB,\(84\)
norm,"\(\left(130, 4\right)\)",\(\frac{67108864}{57}\),\(129\),\(18\),\(18\),\(1\),\(18\),937 KB,144 MB,\(84\)
batch,"\(\left(130, 1\right)\)",\(\frac{67108864}{57}\),\(129\),\(18\),\(28\),\(1\),\(23\),937 KB,144 MB,\(84\)
split,"\(\left(130, 1\right)\)",\(\frac{33554432}{57}\),\(258\),\(18\),\(28\),\(1\),\(23\),1 MB,144 MB,\(84\)
